In [1]:
import os
import torch.utils.data as data_utils
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

In [2]:
### HYPERPARAMETERS ###
dataset_no = 1  # 1 for color, 2 for BnW

if dataset_no==1:
    convergence = 1e-7
    batch_size = 4
    learning_rate = 1e-3
    nodes_l1 = 200  # 64,128,256,512  ,best 512
    nodes_l2 = 100  # 64,128,256,384 ,best 384
    nodes_l3 = 30   # 32,64,128,256, best 256

if dataset_no==2:
    convergence = 1e-7
    batch_size = 128
    learning_rate = 1e-2
    nodes_l1 = 512  # 64,128,256,512  ,best 512
    nodes_l2 = 384  # 64,128,256,384 ,best 384
    nodes_l3 = 256   # 32,64,128,256, best 256

In [3]:
### data loading ###

torch.manual_seed(1)
np.random.seed(0)

if dataset_no ==1:
    data1=np.zeros(shape=(1596,829))
    ji=0
    labels = ['coast','forest','highway','street','tallbuilding']
    for idx, label in enumerate(labels):
        path = 'Features/'+label
        for file in os.listdir(path):
            current = os.path.join(path, file)
            in1=open(current)
            l1 = in1.read().strip().split("\n")
            l2=[]
            for i in l1:
                l2=l2+i.split(" ")
            l2.append(idx)
            l2=np.array(l2)
            l2=np.float_(l2)
            data1[ji]=l2
            ji=ji+1

if dataset_no ==2:
    datafile = 'BnW/7/data.csv'
    data1 = pd.read_csv(datafile, encoding = "UTF-8")
    data1=data1.values
            
np.random.shuffle(data1)
train_test_split = int(0.8*data1.shape[0]) # 80% traindata
data_train = data1[:train_test_split]
data_test  = data1[train_test_split:]

unlabeled_labeled_split = int(0.7*data1.shape[0]) # 70% of traindata as unlabeled data
data_labeled = data1[unlabeled_labeled_split:]
data_train = data_labeled

targets=torch.Tensor(np.ravel(data_train[:,-1]))
features=torch.Tensor(data_train[:,:-1])
targets_t=torch.Tensor(np.ravel(data_test[:,-1]))
features_t=torch.Tensor(data_test[:,:-1])

dataset = data_utils.TensorDataset(features, targets)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
input_dim = data1[:,:-1].shape[1]

In [4]:
### Model definition ###
class autoencoder1(nn.Module):                # autoencoder 1 model
    def __init__(self):
        super(autoencoder1, self).__init__()
        self.encoder1 = nn.Sequential(
            nn.Linear(input_dim, nodes_l1),
            nn.ReLU(True))
        self.decoder1 = nn.Sequential(
            nn.Linear(nodes_l1, input_dim),
            nn.Sigmoid())

    def bottle(self,x):                     # forward pass through encoder only
            x=self.encoder1(x)
            return x

        
    def forward(self, x):                   # forward pass through encoder and decoder
            x = self.encoder1(x)
            x = self.decoder1(x)
            return x

        
class autoencoder2(nn.Module):
    def __init__(self):
        super(autoencoder2, self).__init__()
        self.encoder2 = nn.Sequential(
            nn.Linear(nodes_l1,nodes_l2),
            nn.ReLU(True))
        self.decoder2 = nn.Sequential(
            nn.Linear(nodes_l2,nodes_l1),
            nn.ReLU(True))

    def bottle(self,x):
            x=self.encoder2(x)
            return x

        
    def forward(self, x):
            x = self.encoder2(x)
            x = self.decoder2(x)
            return x

        
class autoencoder3(nn.Module):
    def __init__(self):
        super(autoencoder3, self).__init__()
        self.encoder3 = nn.Sequential(
            nn.Linear(nodes_l2,nodes_l3),
            nn.ReLU(True))
        self.decoder3 = nn.Sequential(
            nn.Linear(nodes_l3,nodes_l2),
            nn.ReLU(True))

    def bottle(self,x):
            x=self.encoder3(x)
            return x

        
    def forward(self, x):
            x = self.encoder3(x)
            x = self.decoder3(x)
            return x

model1 = autoencoder1()
model2 = autoencoder2()
model3 = autoencoder3()


In [5]:
model1.load_state_dict(torch.load('./autoencoder1.pth'))
model2.load_state_dict(torch.load('./autoencoder2.pth'))
model3.load_state_dict(torch.load('./autoencoder3.pth'))


In [6]:
class SAE(nn.Module):
    def __init__(self):
        super(SAE, self).__init__()
        self.encoder1 = nn.Sequential(
            nn.Linear(input_dim, nodes_l1),
            nn.ReLU(True))
        self.encoder2 = nn.Sequential(
            nn.Linear(nodes_l1,nodes_l2),
            nn.ReLU(True))
        self.encoder3 = nn.Sequential(
            nn.Linear(nodes_l2,nodes_l3),
            nn.ReLU(True))
        self.final_layer = nn.Sequential(
            nn.Linear(nodes_l3, 5),
            nn.Softmax(1))

    def forward(self, x):
        x = self.encoder1(x)
        x = self.encoder2(x)
        x = self.encoder3(x)
        x = self.final_layer(x)
        return x

sae_model = SAE()
# classif==sae_model

In [7]:
sae_model.encoder1.load_state_dict(model1.encoder1.state_dict(), strict=True)
sae_model.encoder2.load_state_dict(model2.encoder2.state_dict(), strict=True)
sae_model.encoder3.load_state_dict(model3.encoder3.state_dict(), strict=True)



In [8]:
### Classification ###

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    sae_model.parameters(), lr=learning_rate,momentum=0.9, weight_decay=1e-5)

best_model_cls=sae_model
prev_loss=10
train_cost_history = []
epoch=0

while(True):
    epoch+=1
    for data in dataloader:
        img, clas = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        clas=clas.type(torch.LongTensor)
        # ===================forward=====================
        #print (img.shape)
        output = sae_model(img)
        loss = criterion(output, clas)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    delta_loss = prev_loss - loss.data
    if dataset_no==1:
        if epoch==200:  # convergence criteria gives bad results for dataset 1
            break
    else:
        if delta_loss < convergence: # convergence gives good results for dataset 2
            break
    best_model_cls=sae_model
    prev_loss=loss.data
    train_cost_history.append(loss.data)

    # ===================log========================
    ## train acc ##
    feat=Variable(features)
    output=sae_model(feat)
    targ=targets.type(torch.LongTensor)
    tot=0
    corr=0
    #print (targ[0].item())
    for i in range(targ.shape[0]):
        in1=0
        ma=0
        for j in range(output.shape[1]):
            if (output[i][j].item() > ma):
                ma=output[i][j].item()
                in1=j

        if (targ[i].item() == in1):
            corr=corr+1
        tot=tot+1
    train_acc=corr/tot
    
    ## test acc ##
    feat=Variable(features_t)
    output=sae_model(feat)
    targ=targets_t.type(torch.LongTensor)
    tot=0
    corr=0
    for i in range(targ.shape[0]):
        in1=0
        ma=0
        for j in range(output.shape[1]):
            if (output[i][j].item() > ma):
                ma=output[i][j].item()
                in1=j

        if (targ[i].item() == in1):
            corr=corr+1
#         print (in1,targ[i].item())
        tot=tot+1
    test_acc=corr/tot
    
#     print('epoch {}, loss:{:.4f}, train_accuracy:{:.4f}, test_accuracy:{:.4f}'
#           .format(epoch, loss.data, train_acc, test_acc))
    print('{}, {:.4f}, {:.4f}, {:.4f}'
          .format(epoch, loss.data, train_acc, test_acc))
torch.save(best_model_cls.state_dict(), './sae_model_final_pretrained.pth')


1, 1.8241, 0.2088, 0.2156
2, 1.7372, 0.3152, 0.3344
3, 1.6347, 0.3612, 0.3719
4, 1.5658, 0.4092, 0.4188
5, 1.5589, 0.4196, 0.4281
6, 1.5511, 0.4489, 0.4469
7, 1.5469, 0.4885, 0.4844
8, 1.5460, 0.5198, 0.5156
9, 1.5451, 0.5136, 0.5125
10, 1.5490, 0.5428, 0.5469
11, 1.5485, 0.5574, 0.5625
12, 1.5500, 0.5783, 0.5750
13, 1.5550, 0.5595, 0.5563
14, 1.5524, 0.5762, 0.5781
15, 1.5467, 0.5971, 0.6000
16, 1.5428, 0.6305, 0.6312
17, 1.5438, 0.6430, 0.6406
18, 1.5382, 0.6660, 0.6625
19, 1.5290, 0.6743, 0.6719
20, 1.5282, 0.6722, 0.6750
21, 1.5269, 0.6743, 0.6750
22, 1.5294, 0.6827, 0.6813
23, 1.5262, 0.6848, 0.6844
24, 1.5307, 0.6806, 0.6781
25, 1.5245, 0.6931, 0.6937
26, 1.5370, 0.6889, 0.7000
27, 1.5685, 0.6806, 0.6781
28, 1.5714, 0.6159, 0.6188
29, 1.5679, 0.6722, 0.6781
30, 1.5706, 0.6242, 0.6344
31, 1.5438, 0.6868, 0.7000
32, 1.5231, 0.6931, 0.7094
33, 1.5303, 0.6681, 0.6844
34, 1.5246, 0.6931, 0.7063
35, 1.5350, 0.6973, 0.7156
36, 1.5225, 0.6889, 0.7094
37, 1.5283, 0.6994, 0.7188
38, 1.5306

In [9]:
sae_model.load_state_dict(torch.load('./sae_model_final_pretrained.pth'))

feat=Variable(features)
output=sae_model(feat)
pred = torch.argmax(output, 1)
targ=targets.type(torch.LongTensor)

cm = confusion_matrix(pred.view(-1), targ.view(-1))


In [11]:
labels = ['coast','forest','highway','street','tallbuilding']

print("_ ,",end="")
for i in range(cm.shape[0]):
    print(labels[i],',',end="")
print("")
for i in range(cm.shape[0]):
    print(labels[i],',',end="")
    for j in range(cm.shape[1]):
        print('{:.4f} , '.format(cm[i][j]),end = "")
    print("")

_ ,coast ,forest ,highway ,street ,tallbuilding ,
coast ,103.0000 , 1.0000 , 5.0000 , 1.0000 , 2.0000 , 
forest ,2.0000 , 86.0000 , 1.0000 , 0.0000 , 1.0000 , 
highway ,2.0000 , 0.0000 , 84.0000 , 2.0000 , 1.0000 , 
street ,1.0000 , 0.0000 , 1.0000 , 77.0000 , 0.0000 , 
tallbuilding ,1.0000 , 3.0000 , 0.0000 , 5.0000 , 100.0000 , 
